# Ethereum ECDSA signature playground
## Prepare

To interact with Ethereum network, you need Ethereum node up and running.
You can run both notebook and ganache-cli node emulator by `./start.sh` script.

Or you can run ganache-cli Ethereum emulator in separate terminal
`npx ganache-cli -m "dawn finish orchard pluck festival genuine absorb van bike mirror kiss loop"`
(12 words are the seed passphrase to keep addresses and keys constant)

## Connect to Web3
Now connect to Ethereum provider via web3 RPC

In [1]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.eth.defaultAccount = w3.eth.accounts[0]

Check if web3 up and running

In [2]:
w3.eth.getBlock('latest')

AttributeDict({'number': 4,
 'hash': HexBytes('0xee12fa837bace87afc5fd3972982d4d1fb3afb99f77007b4f6750077b86f0d45'),
 'parentHash': HexBytes('0xbc036b97685ed7c260398412732b053ef1de5612cd89d18efc37fda59101a397'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000080000000000000000000000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000020000000000000000000800000000000000000000000010000000000000000020000000000000000000000000000000000000000000000000040020000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000820000000000000800000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

## Solidity version
Solc compiler has to be installed on your machine. Check solidity version (should match pragma statement in your contract)

In [4]:
import subprocess, re, json
solc = subprocess.Popen(['solc', '--version'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
solc_output = solc.communicate()

m = re.search(r"Version: ([\w\.\+]+)", str(solc_output[0]))
m.group(1)

'0.5.7+commit.6da8b019.Darwin.appleclang'

## Solidity verification contract
The contract which uses ECRECOVER solidity function to recover address from hash of the data and its signature.

In [5]:
contract_source_code = b"""

pragma solidity ^0.5.7;

contract Auth {      
    function verify(address p, bytes32 hash, uint8 v, bytes32 r, bytes32 s) pure public returns(bool) {
        return ecrecover(hash, v, r, s) == p;
    }
}

"""

Actually compile and generate ABI

In [6]:
solc = subprocess.Popen(['solc', '--combined-json', 'bin,abi', '-'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
solc_output = solc.communicate(contract_source_code)

See the contract bytecode

In [7]:
bytecode = json.loads(solc_output[0])['contracts']['<stdin>:Auth']['bin']
bytecode

'608060405234801561001057600080fd5b50610185806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063506535f314610030575b600080fd5b61009d600480360360a081101561004657600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919080359060200190929190803560ff16906020019092919080359060200190929190803590602001909291905050506100b7565b604051808215151515815260200191505060405180910390f35b60008573ffffffffffffffffffffffffffffffffffffffff1660018686868660405160008152602001604052604051808581526020018460ff1660ff1681526020018381526020018281526020019450505050506020604051602081039080840390855afa15801561012d573d6000803e3d6000fd5b5050506020604051035173ffffffffffffffffffffffffffffffffffffffff161490509594505050505056fea165627a7a7230582093a5ddcab62c2e993758d6e42615e75567392c49cb4606fd6ae140e1496c640b0029'

See the contract ABI interfaces

In [8]:
abi = json.loads(solc_output[0])['contracts']['<stdin>:Auth']['abi']
abi

'[{"constant":true,"inputs":[{"name":"p","type":"address"},{"name":"hash","type":"bytes32"},{"name":"v","type":"uint8"},{"name":"r","type":"bytes32"},{"name":"s","type":"bytes32"}],"name":"verify","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"pure","type":"function"}]'

## Deploy
Instantiate contract fabric and deploy the contract on the net

In [9]:
AuthContract = w3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = AuthContract.constructor().transact()
tx_hash

HexBytes('0x77210a6c3bf1cdafbdd90ee35438d63976de78c87ef25f6deb529d079bfac28b')

Get Tx receipt (see contractAddress)

In [10]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x77210a6c3bf1cdafbdd90ee35438d63976de78c87ef25f6deb529d079bfac28b'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x89b17454a67c9f3e0d4fceacb708760277ab09aff5d4f5fd3bd361eece918861'),
 'blockNumber': 5,
 'from': '0x90c08087af274b77516df05952273008fea2c4b9',
 'to': None,
 'gasUsed': 158277,
 'cumulativeGasUsed': 158277,
 'contractAddress': '0xF7a848FFf7a333D53B7F9B0077BD4D031B40A73b',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Initialize contract instance at given address

In [11]:
auth_contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi,
)
auth_contract

## Interact
Call the contract methods (ToDo)

In [12]:
auth_contract.functions.verify(
    '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
    bytes(0),
    1,
    bytes(b'\xff'*32),
    bytes(b'\xff'*32)).call()

False